In [54]:
# Manual-aperture centroid drift logger (main camera) + FITS saving (optional rotation)
import time, csv, pathlib, datetime as dt
import numpy as np
import win32com.client as win32

# ------- User settings -------
EXPOSURE_S       = 0.001                # laser exposure
CADENCE_S        = 10                   # seconds between samples
DURATION_S       = 1 * 60               # total runtime
RINGS            = (10, 4, 2, 0)        # (aperture radius, gap, annulus, plane=0)
REF_X, REF_Y     = 3486.831, 1034.933   # UI centroid (pixels)
SAVE_FITS        = True
ROTATE_FITS_K    = -1                   # -1 = −90°, 0 = none, 1 = +90°, etc.
# --------------------------------

try:
    from astropy.io import fits
    ASTROPY_OK = True
except Exception:
    ASTROPY_OK = False
    SAVE_FITS = False
    print("⚠️  astropy not found — FITS saving disabled. Run: pip install astropy")

# --- Connect to MaxIm ---
cam = win32.Dispatch("MaxIm.CCDCamera")
cam.LinkEnabled = True
if not cam.LinkEnabled:
    raise RuntimeError("Failed to link MaxIm camera")

# --- Enable CCD cooler ---
try:
    cam.CoolerOn = True
    cam.TemperatureSetpoint = -15.0
    print(f"❄️  Cooler ON, target = {cam.TemperatureSetpoint:.1f} °C")
except Exception as e:
    print(f"⚠️  Could not enable cooler: {e}")

def expose_and_wait(exp_s: float):
    cam.Expose(exp_s, True)
    while not cam.ImageReady:
        time.sleep(0.01)

def save_fits_frame(path: pathlib.Path, k90: int):
    """Save the current image as FITS with vertical flip + rotation."""
    arr = np.array(cam.ImageArray, dtype=np.int32)
    arr = np.clip(arr, 0, 65535).astype(np.uint16)
    arr = np.flipud(arr)
    if k90 % 4 != 0:
        arr = np.rot90(arr, k90 % 4)

    hdu = fits.PrimaryHDU(arr)
    hdr = hdu.header
    hdr["DATE-OBS"] = dt.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S")
    hdr["EXPTIME"]  = float(EXPOSURE_S)
    hdr["SEEDX"]    = float(REF_X)
    hdr["SEEDY"]    = float(REF_Y)
    hdr["RADIUS"]   = int(RINGS[0])
    hdr["GAP"]      = int(RINGS[1])
    hdr["ANNULUS"]  = int(RINGS[2])
    hdu.writeto(str(path), overwrite=True)

# --- Output setup ---
run_id  = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
outdir  = pathlib.Path(f"MaxImRun_{run_id}")
outdir.mkdir(exist_ok=True)
csv_path = outdir / "centroid_log.csv"
f = open(csv_path, "w", newline="")
w = csv.writer(f)
w.writerow(["timestamp","frame","seedX","seedY","cx","cy","dX","dY","R","Gap","Ann","fits_file"])

# --- Initial exposure and baseline centroid ---
expose_and_wait(EXPOSURE_S)
doc   = cam.Document
seedX, seedY = float(REF_X), float(REF_Y)
info  = doc.CalcInformation(int(round(seedX)), int(round(seedY)), list(RINGS))
cx0, cy0 = float(info[6]), float(info[7])

print("✅ Connected to MaxIm DL (main camera).")
print(f"🎯 Seed (UI): ({seedX:.3f}, {seedY:.3f}); baseline centroid: ({cx0:.9f}, {cy0:.9f})")
print(f"💾 FITS saving: {SAVE_FITS}  | rotation: {ROTATE_FITS_K*90}°\n")
print("Time     |  # |          cx          cy |          dX           dY | FITS")

# --- Loop ---
t0 = time.time()
next_tick = t0
frame = 0
last_cx, last_cy = cx0, cy0

try:
    while time.time() - t0 < DURATION_S:
        now = time.time()
        if now < next_tick:
            time.sleep(next_tick - now)
        next_tick += CADENCE_S
        frame += 1

        expose_and_wait(EXPOSURE_S)
        doc  = cam.Document
        info = doc.CalcInformation(int(round(last_cx)), int(round(last_cy)), list(RINGS))
        cx, cy = float(info[6]), float(info[7])
        dx, dy = cx - cx0, cy - cy0

        # Save FITS if enabled
        if SAVE_FITS:
            fit_path = outdir / f"frame_{frame:03d}.fit"
            save_fits_frame(fit_path, ROTATE_FITS_K)
            fit_name = fit_path.name
        else:
            fit_name = ""

        ts = dt.datetime.now().strftime("%H:%M:%S")
        print(f"{ts} | {frame:02d} | {cx:13.9f} {cy:13.9f} | {dx:+13.9f} {dy:+13.9f} | {fit_name or '-'}")

        w.writerow([dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    frame, f"{last_cx:.9f}", f"{last_cy:.9f}",
                    f"{cx:.9f}", f"{cy:.9f}", f"{dx:.9f}", f"{dy:.9f}",
                    RINGS[0], RINGS[1], RINGS[2], fit_name])
        f.flush()

        last_cx, last_cy = cx, cy

finally:
    f.close()
    print(f"\n✅ Done. CSV saved: {csv_path}")
    if SAVE_FITS:
        print(f"📁 FITS folder: {outdir}")


❄️  Cooler ON, target = -15.0 °C
✅ Connected to MaxIm DL (main camera).
🎯 Seed (UI): (3486.831, 1034.933); baseline centroid: (3486.823730469, 1034.725219727)
💾 FITS saving: True  | rotation: -90°

Time     |  # |          cx          cy |          dX           dY | FITS
15:25:52 | 01 | 3486.879394531 1034.710937500 |  +0.055664062  -0.014282227 | frame_001.fit
15:26:01 | 02 | 3486.840576172 1034.711181641 |  +0.016845703  -0.014038086 | frame_002.fit
15:26:11 | 03 | 3486.843505859 1034.685302734 |  +0.019775391  -0.039916992 | frame_003.fit
15:26:21 | 04 | 3486.808349609 1034.555908203 |  -0.015380859  -0.169311523 | frame_004.fit
15:26:31 | 05 | 3486.806152344 1034.629760742 |  -0.017578125  -0.095458984 | frame_005.fit
15:26:41 | 06 | 3486.829833984 1034.605224609 |  +0.006103516  -0.119995117 | frame_006.fit
15:26:52 | 07 | 3486.812255859 1034.360717773 |  -0.011474609  -0.364501953 | frame_007.fit

✅ Done. CSV saved: MaxImRun_20251029_152541\centroid_log.csv
📁 FITS folder: MaxImRu

### 📊 Centroid Drift and Equivalent RV Impact (Run: 29 Oct 2025)

**Experiment summary**  
- Exposure = 1 ms | Duration ≈ 1 min  
- CCD cooler = ON (−15 °C) | Rotation = −90°  
- Reference = (3476.849, 1345.589)  
- Camera: **ZWO ASI6200MM Pro** (Sony IMX455, 3.76 µm pixels)  
- Laser diode wavelength λ = 635–660 nm  

---

#### Measured centroid motion
| Axis | Mean (px) | RMS (px) | Max drift (px) | Drift (µm) |
|------|------------|-----------|----------------|-------------|
| ΔX | −0.013 | 0.06 | 0.12 | 0.45 µm |
| ΔY | +0.114 | 0.12 | **0.17** | **0.64 µm** |

*(Pixel → µm conversion:  Δs = Δp × 3.76 µm / px)*

---

#### RV conversion

The wavelength scale per pixel is estimated from resolving power **R = 75 000**  
and sampling of **≈ 3 px per resolution element**:

$$
\Delta\lambda_{\text{px}} = \frac{\lambda}{R \times 3}
$$

For λ = 635–660 nm:

$$
\Delta\lambda_{\text{px}} \approx \frac{(635–660)}{75\,000 \times 3}
\approx (0.0028–0.0029)\,\text{nm px}^{-1}
$$

Using the Doppler relation:

$$
\frac{\Delta v}{c} = \frac{\Delta\lambda}{\lambda}
\quad\Rightarrow\quad
\Delta v = c \times \frac{\Delta\lambda_{\text{px}}}{\lambda}
$$

Hence, for λ ≈ 650 nm:

$$
1\,\text{px} \;\Rightarrow\;
3\times10^8 \times
\frac{0.0029\times10^{-9}}{650\times10^{-9}}
\approx 1.34\times10^3\,\text{m s}^{-1}
$$

So **1 px ≈ 1.34 km s⁻¹** in the dispersion direction.

---

| Drift (px) | Drift (µm) | Δv (km s⁻¹) | Δv (m s⁻¹) |
|-------------|-------------|--------------|--------------|
| 0.17 | 0.64 | 0.23 | **230** |
| 0.12 | 0.45 | 0.16 | 160 |
| 0.01 | 0.038 | 0.013 | 13 |
| 0.001 | 0.0038 | 0.0013 | 1.3 |

---

#### Interpretation
- Peak **Y-axis drift ≈ 0.17 px (0.64 µm)** → ≈ **230 m s⁻¹** equivalent RV shift.  
- Drift dominated along the dispersion (Y) direction.  
- Even **1 µm motion ≈ 0.26 px → ~340 m s⁻¹**, highlighting how sub-micron image stability  
  directly translates to high-precision radial-velocity control.  
- To reach **1 m s⁻¹** RV precision, centroid stability must be **≤ 0.001 px (~4 nm)**.
on stability (≤ 0.001 px) is required for **~1 m s⁻¹** RV precision.
he stability goal for high-resolution spectrograph feedback control.
¹ RV precision**, consistent with high-resolution EPRV goals.
 m s⁻¹) using temperature-f
d environmental stabilization loops.


In [52]:
import win32com.client, time

# --- Connect to MaxIm DL ---
cam = win32com.client.Dispatch("MaxIm.CCDCamera")
cam.LinkEnabled = True
print("✅ Connected to MaxIm DL (main camera)")

# --- Single short exposure ---
EXPOSURE_S = 0.001
cam.Expose(EXPOSURE_S, True)
while not cam.ImageReady:
    time.sleep(0.05)

# --- Access the current image document ---
doc = cam.Document

# Pick a reference pixel near the bright spot (or center)
seed_x, seed_y = 3486.901, 1034.624  # replace with your target area if known

# Rings: (aperture radius, gap, annulus thickness, plane=0)
rings = (10, 4, 2, 0)

# --- Calculate centroid from that region ---
info = doc.CalcInformation(seed_x, seed_y, rings)
cx, cy = float(info[6]), float(info[7])

print("Centroid position (from Info panel):")
print(f"  X = {cx:.8f} px")
print(f"  Y = {cy:.8f} px")


✅ Connected to MaxIm DL (main camera)
Centroid position (from Info panel):
  X = 3486.85278320 px
  Y = 1034.46606445 px


Do we know the sampling of our spectrograph — how many detector pixels correspond to one resolution element? I’ve seen 2–3 px/element as typical, but I’m not sure what value our setup uses since we only have the grating installed.